In [1]:
 # This is Notebook for File Type detection using convolutional neural network on TREC-DD dataset
 # The data used is in 3 part of 256 byte frequencies format, first 256, middle, last 256
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import pickle
from utility import *
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
tf.logging.set_verbosity(tf.logging.INFO)

In [16]:
def cnn_model_fn(features, labels, mode, params):
  """Model function for CNN."""
  
  config = params

  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  
  input_layer = tf.reshape(features["x"], [-1, 256, 3, 1])
  print('input shape:',input_layer.shape)
    
  #input_dropout = tf.layers.dropout(
  #    inputs=input_layer, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)


  # kernel_size specifies [width, height]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[config['kernel1_width'], 1],
      padding="same",
      activation=None)
  print('conv1 shpae:',conv1.shape)  
  normal_1 = tf.layers.batch_normalization(conv1, training=mode == tf.estimator.ModeKeys.TRAIN)
  relu1 = tf.nn.relu(normal_1)
  pool1 = tf.layers.max_pooling2d(inputs=relu1, pool_size=[config['pool1_width'], 1], strides=[config['pool1_stride'],1])
  print('pool1 shape:',pool1.shape)
  dropout1 = tf.layers.dropout(
      inputs=pool1, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  
 
  conv2 = tf.layers.conv2d(
      inputs=dropout1,
      filters=64,
      kernel_size=[config['kernel2_width'], 1],
      padding="same",
      activation=None)
  print('conv2 shpae:',conv2.shape)
  normal_2 = tf.layers.batch_normalization(conv2, training=mode == tf.estimator.ModeKeys.TRAIN)  
  relu2 = tf.nn.relu(normal_2)
  pool2 = tf.layers.max_pooling2d(inputs=relu2, pool_size=[config['pool2_width'], 1], strides=[config['pool2_stride'],1])
  print('pool2 shape:',pool2.shape)
  dropout2 = tf.layers.dropout(
      inputs=pool2, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)  

  
  conv3 = tf.layers.conv2d(
      inputs=dropout2,
      filters=128,
      kernel_size=[config['kernel2_width'], 1],
      padding="same",
      activation=None)
  print('conv3 shpae:',conv3.shape)
  normal_3 = tf.layers.batch_normalization(conv3, training=mode == tf.estimator.ModeKeys.TRAIN)  
  relu3 = tf.nn.relu(normal_3)
  pool3 = tf.layers.max_pooling2d(inputs=relu3, pool_size=[config['pool2_width'], 1], strides=[config['pool2_stride'],1])
  print('pool3 shape:',pool3.shape)
  dropout3 = tf.layers.dropout(
      inputs=pool3, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  '''
  conv4 = tf.layers.conv2d(
      inputs=dropout3,
      filters=256,
      kernel_size=[config['kernel2_width'], 1],
      padding="same",
      activation=None)
  print('conv4 shpae:',conv4.shape)
  normal_4 = tf.layers.batch_normalization(conv4, training=mode == tf.estimator.ModeKeys.TRAIN)  
  relu4 = tf.nn.relu(normal_4)
  #pool4 = tf.layers.max_pooling2d(inputs=relu4, pool_size=[config['pool2_width'], 1], strides=[config['pool2_stride'],1])
  #print('pool4 shape:',pool4.shape)
  dropout4 = tf.layers.dropout(
      inputs=relu4, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
    
   
  conv5 = tf.layers.conv2d(
      inputs=dropout4,
      filters=128,
      kernel_size=[config['kernel2_width'], 1],
      padding="same",
      activation=None)
  print('conv5 shpae:',conv5.shape)
  normal_5 = tf.layers.batch_normalization(conv5, training=mode == tf.estimator.ModeKeys.TRAIN)  
  relu5 = tf.nn.relu(normal_5)
  #pool5 = tf.layers.max_pooling2d(inputs=relu5, pool_size=[config['pool2_width'], 1], strides=[config['pool2_stride'],1])
  #print('pool5 shape:',pool5.shape)
  dropout5 = tf.layers.dropout(
      inputs=relu5, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  '''
  
  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 64, 1, 64]
  # Output Tensor Shape: [batch_size, 64 * 1* 64]]
  dropout3_shape = dropout3.shape
  pool3_flat = tf.reshape(dropout3, [-1, dropout3_shape[1] * dropout3_shape[2] * dropout3_shape[3]])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 64 * 1* 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool3_flat, units=config['dense_units'], activation=None)
  dense_norm = tf.layers.batch_normalization(dense, training=mode == tf.estimator.ModeKeys.TRAIN)  

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense_norm, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 93]
  logits = tf.layers.dense(inputs=dropout, units=config['nclasses'])

    
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=config['nclasses'])
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
unused_argv = ['pilot_1d_cnn_newAPI.py', 'temp_data.csv', 'model/', 1024, 512, 256]
filename = unused_argv[1]

if os.path.isfile('ft_to_idx.npy') and os.path.isfile('idx_to_ft.npy') and os.path.isfile('nclasses.npy') and os.path.isfile('group_data.npy'):
    ft_to_idx = np.load('ft_to_idx.npy')
    ft_to_idx = ft_to_idx.item()
    idx_to_ft = np.load('idx_to_ft.npy')
    idx_to_ft = idx_to_ft.item()
    nclasses = np.load('nclasses.npy')
    #f = open('group_data.pkl','r')
    #group_data = pickle.load('group_data.pkl')
    #group_data = np.load('group_data.npy') 
    group_data = np.load('toy_data.npy') # for proof of algo purpose, real use case should use the above line
    group_data = group_data.item()

else:
    ft_to_idx, idx_to_ft, nclasses, group_data = prepare_file(filename)
    np.save("ft_to_idx", ft_to_idx)
    np.save("idx_to_ft", idx_to_ft)
    np.save("nclasses", nclasses)
    #f = open('group_data.pkl','w')
    #pickle.dump(group_data, f)
    np.save("group_data", group_data)

#gen train set
cnt = map(lambda x: len(x),group_data.values())
percentile_40 = np.percentile(cnt, 40)
train, dev ,test = train_dev_test_split(group_data, proportion = [0.6,0.2], thre = percentile_40) # should be 1000 or so
train_data, train_labels = gen_feed(train, ft_to_idx, upper_limit=12000)
del train,group_data
dev_data, dev_labels = gen_feed(dev, ft_to_idx, upper_limit=3000)
del dev
test_data, test_labels = gen_feed(test, ft_to_idx, upper_limit=3000)
del test
train_data = train_data.astype(np.float32) 
dev_data = dev_data.astype(np.float32)
test_data = test_data.astype(np.float32) 

scaler = StandardScaler()
train_data_norm = scaler.fit_transform(train_data)
dev_data_norm = scaler.transform(dev_data)
test_data = scaler.transform(test_data)



In [17]:
from itertools import product
dense_units = [128]#256,512]# 
batch_sizes = [128]# 64,256,512]
dropouts = [0.1]#0.4,0.2, 0.6]
kernel_width = [2,16]#,8]#4,5,7]
pool_width = [2,4]
pool_stride = [2]#,4] 4 will be too large


#reset config, defined in previous block
config = {}
config['nclasses'] =  len(np.unique(train_labels))
results = []
for dense_unit,batch_size,dropout,kernel_width,pool_width,pool_stride in product(dense_units,batch_sizes,dropouts,kernel_width,pool_width,pool_stride):
    config['dropout'] = dropout
    config['dense_units'] = dense_unit
    config['kernel1_width'] = kernel_width
    config['kernel2_width'] = kernel_width
    config['pool1_width'] = pool_width
    config['pool2_width'] = pool_width
    config['pool1_stride'] = pool_stride
    config['pool2_stride'] = pool_stride
    print ('\n----setting----')
    print('{0:12} ==> {1:5d}'.format('batch_size', batch_size))
    for k,v in config.iteritems():
        print('{0:12} ==> {1:5f}'.format(k, v))  
    # Create the Estimator
    cnn_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=None, params=config)

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=5000) 

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=batch_size,
      num_epochs=3,
      shuffle=True)

    steps = train_data.shape[0]/batch_size
    cnn_classifier.train(
      input_fn=train_input_fn,
      steps=None, # 60000
      hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": dev_data},
        y=dev_labels,
        num_epochs=1,
        shuffle=False)

    eval_results = cnn_classifier.evaluate(input_fn=eval_input_fn)
    print ('\n----setting----')
    print('{0:12} ==> {1:5d}'.format('batch_size', batch_size))
    for k,v in config.iteritems():
        print('{0:12} ==> {1:5f}'.format(k, v))  
    print ('----performance----')
    print(eval_results)
    print ('\n')
    results.append((eval_results['accuracy'], batch_size, dropout, dense_unit, kernel_width, pool_width, pool_stride))
    


----setting----
batch_size   ==>   128
dense_units  ==> 128.000000
kernel2_width ==> 2.000000
dropout      ==> 0.100000
pool2_width  ==> 2.000000
pool1_stride ==> 2.000000
nclasses     ==> 71.000000
pool2_stride ==> 2.000000
kernel1_width ==> 2.000000
pool1_width  ==> 2.000000
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11a989590>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpgYx1Rp', '_save_summary_steps': 100}
input shape: (?, 256, 3, 1)
conv1 shpae: (?, 256, 3, 32)
pool1 shape: (?, 128, 3, 32)
conv2 shpae: (?, 

In [18]:
results = sorted(results, key=lambda x: x[0], reverse=True)
print (results)

[(0.27266961, 128, 0.1, 128, 2, 4, 2), (0.17060792, 128, 0.1, 128, 16, 4, 2), (0.14745374, 128, 0.1, 128, 2, 2, 2), (0.12764758, 128, 0.1, 128, 16, 2, 2)]


In [19]:
best_param = results[0]
batch_size = best_param[1]
config['dropout'] = best_param[2]
config['dense_units'] = best_param[3]
config['kernel1_width'] = best_param[4]
config['kernel2_width'] = best_param[4]
config['pool1_width'] = best_param[5]
config['pool2_width'] = best_param[5]
config['pool1_stride'] = best_param[6]
config['pool2_stride'] = best_param[6]

# Create the Estimator
cnn_classifier = tf.estimator.Estimator(
model_fn=cnn_model_fn, model_dir=None, params=config)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=5000) 

#make full training data
train_full_data = np.vstack((train_data,dev_data))
train_full_labels = np.hstack((train_labels,dev_labels))
train_full_data = scaler.fit_transform(train_full_data)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_full_data},
  y=train_full_labels,
  batch_size=batch_size,
  num_epochs=3,
  shuffle=True)

steps = train_full_data.shape[0]/batch_size
cnn_classifier.train(
  input_fn=train_input_fn,
  steps=None, # 60000
  hooks=[logging_hook])

pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    num_epochs=1,
    shuffle=False)
predictions = cnn_classifier.predict(input_fn=pred_input_fn)
predictions = list(p["classes"] for p in predictions)

print ('----Best setting----')
print('{0:12} ==> {1:5d}'.format('batch_size', batch_size))
for k,v in config.iteritems():
    print('{0:12} ==> {1:5f}'.format(k, v))  
print ('\n')

target_names = np.unique(test_labels)
target_names = np.sort(target_names)
target_names = [idx_to_ft[i] for i in target_names]
print (classification_report(test_labels, predictions, target_names=target_names))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1193c4350>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpv4NXhM', '_save_summary_steps': 100}
input shape: (?, 256, 3, 1)
conv1 shpae: (?, 256, 3, 32)
pool1 shape: (?, 127, 3, 32)
conv2 shpae: (?, 127, 3, 64)
pool2 shape: (?, 62, 3, 64)
conv3 shpae: (?, 62, 3, 128)
pool3 shape: (?, 30, 3, 128)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpv4NXhM/model.ckpt.
INFO:tensorflow:prob

In [20]:
from sklearn.metrics import accuracy_score
print (accuracy_score(test_labels, predictions,normalize=True))

0.37538288209
